In [22]:
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

In [23]:
train_data = pd.read_csv("./smile_description_train.csv")
test_data = pd.read_csv("./smile_description_test.csv")

In [24]:
# Replacing Null values of train and test data with 0 
train_data = train_data.fillna(0)
test_data = test_data.fillna(0)

In [25]:
# Split into X and y
X = train_data.drop("label",axis=1)
y = train_data["label"]

In [26]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 100)
transformed_train_data = pca.fit_transform(X.drop("Assay_id",axis=1))
transformed_test_data =  pca.fit_transform(test_data.drop("Assay_id",axis=1))

In [27]:
transformed_train_data = pd.DataFrame(data=transformed_train_data,columns=pca.get_feature_names_out())
transformed_test_data =  pd.DataFrame(data=transformed_train_data,columns=pca.get_feature_names_out())

In [28]:
transformed_train_data["Assay_id"] = X["Assay_id"]
transformed_test_data["Assay_id"] = test_data["Assay_id"]

In [29]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [30]:
def create_submission(predict,filename):
    sub_file = pd.read_csv("./data/sample_submission.csv")
    sub_file["Predicted"] = predict
    sub_file.to_csv(filename,index=False)
    print(filename," Created")

In [31]:
xgb_c = xgb.XGBClassifier(random_state=42,n_estimators=600)

In [61]:
def f1_score_xg_boost(X,y,estimator,learning_rate):
    skf = StratifiedKFold(n_splits=5,random_state=42,shuffle=True)
    cv =  skf.get_n_splits(X, y)
    scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}
    cv_results = cross_validate(xgb.XGBClassifier(random_state=42,n_estimators=estimator,eta=learning_rate), X, y, cv=cv,scoring=scoring,verbose=3,n_jobs=-1)
    print("F1 score with ",sum(cv_results["test_f1_score"])/5)
    print("Accuracy score with ",sum(cv_results["test_accuracy"])/5)

In [14]:
import numpy as np
def get_count_of_ones_and_twos(predict):
    print("Number of predicted ones",np.count_nonzero(predict==1))
    print("Number of predicted twos",np.count_nonzero(predict==2))

In [ ]:
f1_score_xg_boost(X,y,2000,0.3)
"""
n_estimators = 700
learning_rate = 0.35
pca = 30 components
F1 score with  0.9472493912457081
Accuracy score with  0.9082816785922123
-------

"""

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


In [57]:
xgb_c = xgb.XGBClassifier(random_state=42,n_estimators=2300,eta=0.3)

In [58]:
xgb_c.fit(X,y)
predict = xgb_c.predict(test_data)
predict_real = label_encoder.inverse_transform(predict)

In [59]:
get_count_of_ones_and_twos(predict_real)

Number of predicted ones 1378
Number of predicted twos 9616


In [60]:
create_submission(predict_real,"submission_1_mar_8.csv")

submission_1_mar_8.csv  Created
